In [5]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.alert import Alert
import re
from selenium.webdriver.support.ui import Select
from time import sleep
from random import uniform

In [6]:
dir_for_shinhan = 'C:/Users/shic/Desktop/shinhan_simon/chromedriver/chromedriver.exe'
dir_for_seunghun = 'C:/Users/rsh15/Desktop/seunghuni/selenium/chromedriver.exe'

In [7]:
# chrome driver 지정해주고, 열어서 법원경매 사이트 접속
driver = webdriver.Chrome(dir_for_shinhan)
driver.get(url = 'http://www.courtauction.go.kr/RealUtilMaeTong.laf')

# 시/도 옵션 목록 가져오고, 리스트로 만들기
soup = BeautifulSoup(driver.page_source,'html.parser')
big_regions = soup.find('select',{'id':'idSidoCode'}).find_all('option',{'value':re.compile('\d')})
big_regions = [k.text for k in big_regions]

# 소재지 버튼 클릭
driver.find_element_by_id('idBubwLocGubun2').click()

# 창 maximize
driver.maximize_window()

In [8]:
big_regions

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

각 시/도의 시/군/구별로 월별 데이터를 가져옴.   
개별 데이터를 가져오는 것은 시간이 오래 걸리지 않으나, 경우의 수가 축적되며 상당히 오랜 시간이 걸림

In [9]:
regions_final = []
for sido in big_regions:
    sleep(uniform(0,2))
    print(sido,'start.')
    # 한 시/도의 데이터프레임을 하나의 리스트에 담는다.
    big_regions_list = []
    
    # 시/도 옵션 입력란 찾아서 넣어준다.
    select1= driver.find_element_by_id('idSidoCode')
    select1.send_keys(sido)
    
    sleep(uniform(0,1))
    
    # 시/도 옵션에 따라 바뀌는 시/군/구 데이터
    soup = BeautifulSoup(driver.page_source,'html.parser')
    small_regions = soup.find('select',{'id':'idSiguCode'}).find_all('option',{'value':re.compile('\d')})
    small_regions = [k.text.replace('\n','') for k in small_regions]
    
    # 시/도 마다 시/군/구 데이터로 Loop
    for sigu in small_regions:
        select_sido = driver.find_element_by_id('idSidoCode')
        select_sido.send_keys(sido)
        
        select_sigu = driver.find_element_by_id('idSiguCode')
        select_sigu.send_keys(sigu)
        
        for year_idx in range(9,-1,-1):
            sleep(uniform(0,2))
            for month_idx in range(11,12):
                if (year_idx==0 and month_idx > 6):
                    break
                
                # send start year_month
                select_start_year = Select(driver.find_element_by_id('idSelStartYear'))
                select_start_year.select_by_index(year_idx)
                
                select_start_month = Select(driver.find_element_by_id('idSelStartMnth'))
                select_start_month.select_by_index(month_idx)

                # send enddd year_month
                select_enddd_year = Select(driver.find_element_by_id('idSelEndYear'))
                select_enddd_year.select_by_index(year_idx)
                
                select_enddd_month = Select(driver.find_element_by_id('idSelEndMnth'))
                select_enddd_month.select_by_index(month_idx)
                
                # 조회버튼 클릭
                driver.find_element_by_class_name('Ltbl_srch_btn').click()

                # page_source 가져와서 테이블 파싱
                soup = BeautifulSoup(driver.page_source,'html.parser')
                parse_table = pd.read_html(driver.page_source)[0]

                # parse_table에 시/도, 시/군/구 컬럼 추가
                parse_table['sido'] = sido
                parse_table['sigu'] = sigu
                parse_table['year'] = year_idx
                parse_table['month'] = month_idx
        
                # 시/도 리스트에 데이터프레임 추가
                big_regions_list.append(parse_table)
    print(sido,'complete.',len(small_regions),'small countries.')
    
    # 한 시/도 Loop가 끝나면 최종 리스트에 추가
    regions_final.append(big_regions_list)

서울특별시 start.
서울특별시 complete. 25 small countries.
부산광역시 start.
부산광역시 complete. 17 small countries.
대구광역시 start.
대구광역시 complete. 8 small countries.
인천광역시 start.
인천광역시 complete. 13 small countries.
광주광역시 start.
광주광역시 complete. 5 small countries.
대전광역시 start.
대전광역시 complete. 5 small countries.
울산광역시 start.
울산광역시 complete. 5 small countries.
세종특별자치시 start.
세종특별자치시 complete. 1 small countries.
경기도 start.
경기도 complete. 58 small countries.
강원도 start.
강원도 complete. 18 small countries.
충청북도 start.
충청북도 complete. 16 small countries.
충청남도 start.
충청남도 complete. 20 small countries.
전라북도 start.
전라북도 complete. 16 small countries.
전라남도 start.
전라남도 complete. 22 small countries.
경상북도 start.
경상북도 complete. 25 small countries.
경상남도 start.
경상남도 complete. 28 small countries.
제주특별자치도 start.
제주특별자치도 complete. 2 small countries.


In [10]:
bind_list = [pd.concat(dd,axis=0) for dd in regions_final]
df_court  = pd.concat(bind_list,axis=0)
df_court

,물건용도,물건용도.1,경매건수,매각건수,감정가(단위:원),매각가(단위:원),매각율,매각가율,sido,sigu,year,month
0,아파트,아파트,66,21,29300000000,21535494600,31.8%,73.5%,서울특별시,강남구,9,11
1,"단독주택,다가구주택",단독주택,3,1,1229730990,1069999900,33.3%,87%,서울특별시,강남구,9,11
2,"단독주택,다가구주택",다가구주택,0,0,0,0,0%,0%,서울특별시,강남구,9,11
3,"단독주택,다가구주택",겸용,0,0,0,0,0%,0%,서울특별시,강남구,9,11
4,"단독주택,다가구주택",소계,3,1,1229730990,1069999900,33.3%,87%,서울특별시,강남구,9,11
...,...,...,...,...,...,...,...,...,...,...,...,...
14,"상가,오피스텔,근린시설",근린시설,0,0,0,0,0%,0%,제주특별자치도,제주시,1,11
15,"상가,오피스텔,근린시설",겸용,1,0,0,0,0%,0%,제주특별자치도,제주시,1,11
16,"상가,오피스텔,근린시설",소계,1,0,0,0,0%,0%,제주특별자치도,제주시,1,11
17,기타,기타,0,0,0,0,0%,0%,제주특별자치도,제주시,1,11


In [11]:
df_court.columns = ['물건용도_대분류','물건용도_소분류','경매건수','매각건수','감정가',
                    '매각가','매각율','매각가율','시/도','시/군/구','시기_년','시기_월']

In [12]:
df_court.to_csv('C:/Users/shic/Desktop/shinhan_simon/google_drive/crawler_data/court/df_court_for_december.csv',index=0)